[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1rDXX2GFZwULjMk-JJdU2k9lSYGti7M-f?usp=sharing)
# <strong> Investment Management 1</strong>
---
#<strong> Sharpe Ratio example</strong>

In 1966, William Sharpe introduced the so-called <a href="https://web.stanford.edu/~wfsharpe/art/sr/sr.htm"><em>reward-to-variability ratio</em></a>. While, by Prof. Sharpe's own admission, the "measure has gained considerable popularity, the name has not." For years to come, researchers have termed the original version of the measure the Sharpe Index, the Sharpe Measure, or the Sharpe Ratio (Morningstar). We will use the latter term in this tutorial. 

The Sharpe Ratio is a risk-adjusted measure of investment return that uses standard deviation to represent risk, and it is one of the most popular risk-adjusted performance measures in finance. In its original form, the Sharpe Ratio is calculated as the expected excess return on an asset over the risk-free rate divided by the standard deviation of the excess return:

&nbsp;

$$Sharpe Ratio  = \frac{E|R_p - R_{rf}|}{\sqrt{var(R_p - R_{rf})}} \times \sqrt{N}$$

&nbsp;

where $R_{rf}$ is the benchmark/ risk-free return, $R_p$ is the portfolio return, $𝑁$ is the number of sampling periods in a year. It is common to use the <a href="https://fred.stlouisfed.org/series/TB3MS"> three-month T-bill rate</a> as the risk-free interest rate. The portfolio return and risk-free rate can be observed at any frequency (e.g., daily, weekly, monthly, etc), as long as they are consistent for both measures. For instance, Morningstar calculates a monthly $Sharpe Ratio$ and then annualises it to put the number in a more useful one-year context, as follows:

&nbsp;

$$Sharpe Ratio_{AN} = Sharpe Ratio_M \times \sqrt{12}$$

&nbsp;

We will follow Morningstar's approach in this tutorial. Also, while the Sharpe ratio is typically calculated for a portfolio of assets, in this tutorial we will use stocks instead of portfolios.

The higher the Sharpe Ratio, the better the asset’s historical risk-adjusted performance. It is often used to compare and rank portfolio managers with similar strategies on the basis of excess return each portfolio achieved per unit of total risk.

## 1. Importing required libraries and data

We start by importing the required Python modules/libraries. We further import monthly stock prices for two S&P 500 firms and the 3-month T-bill rate for the 2016-2020 period.

In [ ]:
# First, we import the required modules: pandas, numpy and matplotlib (for visualisations)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Select "ggplot" as the style for Matplotlib plots
# (see https://matplotlib.org/3.1.1/gallery/style_sheets/style_sheets_reference.html)
plt.style.use('ggplot')
%matplotlib inline

# To enable the Colab extension that renders pandas dataframes into interactive tables, we will use:
%load_ext google.colab.data_table


The necessary data for this tutorial are in the <a href="https://github.com/mscouse/TBS_investment_management"> course GitHub repository</a> (see the "1_labs_introduction" folder). You can either clone an entire GitHub repository to your Colab environment or access individual data files using their raw links.
We will follow the latter approach, as we only need to access 2 files in the course repository. See the "PM_labs_part_4" notebook for alternative ways to upload the data.

To load individual files directly from GitHub using thier raw links:

1.   click on the file in the repository;
2.   click on `View Raw`;
3.   copy the URL of the raw file,
4.   use this URL as the location of your file (see sample code below) 

```
# step 1: store the link to your dataset as a string titled "url"
url="https://raw.githubusercontent.com/mscouse/TBS_investment_management/main/1_labs_introduction/stock_prices_1.csv"

# step 2: Load the dataset into pandas. The dataset is stored as a pandas dataframe "df".
df = pd.read_csv(url, parse_dates=['date'], index_col=['date']).dropna()
```

The dataset will be stored as a Pandas Dataframe "df". Note that the data files we deal with in this example contain dates in the first column, titled `'date'`. Therefore, we parse the dates using `parse_dates` and set the date column to be the index of the dataframe (using the `index_col` parameter). We further drop all the missing values by using the `.dropna()` method. 


In [ ]:
# Fetching necessary data from GitHub
# "stock_prices" dataframe contains stock prices on AAPL and JPM
# "tbill_rate" dataframe cotains monthly 1-month Tbill rates


# step 1: store the link to your dataset as a string titled "url"
url_stocks = "https://raw.githubusercontent.com/mscouse/TBS_investment_management/main/1_labs_introduction/stock_prices.csv"
url_tbill = "https://raw.githubusercontent.com/mscouse/TBS_investment_management/main/1_labs_introduction/FF_factors.csv"


"""
step 2: Load the datasets into pandas. The datasets are stored as pandas dataframes
"stock_prices" and "tbill_rate". To make things more interesting, the "stock_prices" 
dataset contains daily prices, whereas the "tbill_rate" contains monthly data. Note that
the dates in the "stock_prices" file we load from GitHub are in the ddmmyyyy format, whereas Pandas
uses the yyyymmdd format instead. Therefore, we convert them using the "datetime" method:
"""
stock_prices = pd.read_csv(url_stocks, index_col=['date']).dropna()
stock_prices.index = pd.to_datetime(stock_prices.index, dayfirst=True)

"""
Also note that the dates in the "FF_factors" file we load from GitHub are initially just integers in the yyyymm format.
Therefore, we convert the index to a datetime, where the date is the last day of the associated month index:
"""
tbill_rate = pd.read_csv(url_tbill, index_col=['date']).dropna()
tbill_rate.index = pd.to_datetime(tbill_rate.index, format='%Y%m') + pd.offsets.MonthEnd()

## 2. First look at the data

Our second task is to explore the loaded data files. Note that for most data-related tasks we use `pandas`, which is an open-source package for data analysis in Python. Pandas is a handy toolkit for working with real world data that often have a tabular structure, such as series of stock prices.

The fundamental `pandas` data structures are `DataFrames` and `Series`. 

Pandas `DataFrame` is a 2-dimensional data structure used for storing and mainpulating tabular data, with rows and columns. You can think of pandas `DataFrame` as a programmable Excel worksheet.

Pandas `Series` is a 1-dimensional data structure used for storing and manipulating a vector of values. Pandas `Series` can be thought of as a list or a column of numerical values in Excel. Any row or column in a Pandas `DataFrame` is, therefore, a Pandas `Series`. We refer to an individual column (`series`) within the dataframe as:

```
df['column_name']
```
See `pandas` <a href="https://pandas.pydata.org/docs/index.html">official documentation</a> for more information.

In [ ]:
# Display summary for the stock_prices dataframe
print('Stock prices\n')
stock_prices.info()
print(stock_prices.head())

# Display summary for the tbill_rate dataframe
print('\nTreasury-bill rates \n')
tbill_rate.info()
print(tbill_rate.head())

## 3. Compute descriptive statistics for daily stock prices

The next step is to compute and examine the basic descriptive statistics for the stock prices. 

With `pandas`, different summary statistics can be applied to columns with numerical data (pandas `series`) or the entire `dataframe`. Operations in general exclude missing data and operate across rows by default. Below are several examples of useful statistics.

**Mean**
```
df['column_name'].mean()
```

**Median:** In this example, the statistic applied to multiple columns of a DataFrame
```
df['column_name_1', 'column_name_2'].median()
```

**Standard deviation**
```
df['column_name'].std()
```
**Aggregating summary statistics:** The aggregating statistic can be calculated for multiple columns at the same time using the `describe` function
```
df.describe()
```



In [ ]:
# compute and store the average stock price for in the 'JPM' column of the stock_prices dataframe 
jpm_mean = stock_prices['JPM'].mean()
print(jpm_mean)

# compute basic descriptive statistics for all columns of the stock_prices dataframe
stock_prices.describe()


## 4. Visualise daily stock prices

The next step is to visualise the data. The pyplot module from matplotlib we imported earlier is the stateful interface of matplotlib. Almost all functions in that module, such as plt.plot(), would either apply to an existing current figure (and its axes), or create them anew if none exist. 

Matplotlib also allows saving figures in a wide range of formats. Saving a figure can be done using the `savefig()` command. You can also adjust the resolution (dpi) of your saved figure. To save the figure as a PNG file, you can use the script below. On the left side of colab interface, there is a "Files/ Folder" tab. You can find all the files you saved there.
```
plt.savefig('sample_figure.png', dpi=300)
```


In [ ]:
# visualise stock_prices
stock_prices.plot(title='Daily Stock Prices', subplots=True, figsize=(10, 8))

# saving the figure
plt.savefig('stock_prices.png', dpi=300)

## 5. Examine and visualise values for Treasury-bill security

We next take a closer look at the available Treasury-bill rates and repeat steps (3)-(4) above.

In [ ]:
# plot the benchmark_data
tbill_rate['RF'].plot(title='Monthly T-bill rate', figsize=(10, 8))

# summarize the benchmark_data
tbill_rate['RF'].describe()

## 6. Compute monthly stock returns

The Sharpe Ratio is computed as the expected excess return on an asset over the risk-free rate divided by the standard deviation of the excess return. However, our dataset contains historical prices for each stock, not the returns. To calculate the returns, we need to calculate the percentage change in stock prices from one day to the next.

Furthermore, as we follow Morningstar's approach to calcuating the Sharpe ratio, we need to convert daily returns into monthly returns. As a reminder, Morningstar calculates a monthly $Sharpe Ratio$ and then annualises it to put the number in a more useful one-year context, as follows:

&nbsp;

$$Sharpe Ratio_{AN} = Sharpe Ratio_M \times \sqrt{12}$$

&nbsp;

Resampling is very easy with `pandas`. `Pandas` has a simple and efficient function `resample()` for performing resampling operations (e.g., converting daily data into monthly data). Data resampling is very common in financial applications. See an examples below:

```
stock_monthly = df['stock_daily'].resample('M')
```

The resample() method can be used directly from DataFrame objects. We also apply the Pandas `dataframe.ffill()` function to fill the missing value in the dataframe.


In [ ]:
# step 1: resample daily stock prices into monthly
stock_prices_monthly = stock_prices.resample('M').ffill()
stock_prices_monthly.head()

In [ ]:
# step 2: calculate monthly stock returns using the .pct_change() function 
stock_returns = stock_prices_monthly.pct_change()

# plot the calculated monthly stock returns; we can plot the last 60 monthly 
# stock returns by selecting the last 60 rows/months of the dataframe as [-60:]
stock_returns[-60:].plot(title='Monthly Stock Returns', subplots=True, figsize=(10, 8))

# summarise the monthly returns
stock_returns.describe()


## 7. Merging the dataframes with stock prices and T-bill rates

For standard database joins between Pandas DataFrame or Series objects, we can use function `merge()`. The join is done on DataFrame columns or indexes. If joining columns on columns, the DataFrame indexes will be ignored. Otherwise, if joining indexes on indexes, or indexes on a column or columns, the index will be passed on. 

Furthermore, when gluing together multiple DataFrames, you have a choice of how to handle the remaining axes - other than the one being concatenated. This can be done in the following two ways:

1. Take the union of them all: use `join='outer'` argument. This is the default option as it results in zero information loss.

2. Take the intersection only: use the `join='inner'` argument.

Putting it all together, to merge the two dataframes we work with, we use:

```
merged_dataframe = pd.merge(left_dataframe, right_dataframe, on='date',   how='inner')
```
OR
```
merged_dataframe = left_dataframe(right_dataframe, how='inner', on='date')
```

For more tips on merging, joining, concatenating Pandas dataframes, see <a href="https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html">pandas documentation</a>.

In [ ]:
# Merge the two dataframes we work with
merged_data = pd.merge(tbill_rate, stock_returns, on='date', how='inner').fillna(0)

# Drop redundant columns, such as MRK-RF, SMB and HML
merged_data_tidy = merged_data.drop(['MRK-RF', 'SMB', 'HML'], axis=1)
merged_data_tidy.head()

## 8. Calculating stock excess returns
Next, we need to calculate the relative performance of stocks vs. the Treasury-bill asset. This is calculated as the difference in returns between "stock_returns" and the "RF" series in the `merged_data_tidy` dataframe for each month. The excess returns will be stored as new pandas series within the same dataframe:
```
merged_data_tidy['new_series'] = ...
```
This code will create a new pandas dataseries `new_series` in the existing `merged_data_tidy` dataframe.

In [ ]:
# calculate excess stock returns by adding several new series into the "merged_data_tidy" dataframe
merged_data_tidy['JPM_exc'] = merged_data_tidy['JPM']-merged_data_tidy['RF'] 
merged_data_tidy['MSFT_exc'] = merged_data_tidy['MSFT']-merged_data_tidy['RF'] 

# plot the excess returns for one of the stocks
merged_data_tidy['JPM_exc'].plot(title="JPM excess returns",  figsize=(10, 8))

# summarize the excess returns
merged_data_tidy.describe()

## 9. Computing the Sharpe Ratio

We are now ready to put it all together and compute the Sharpe Ratios for our chosen stocks. We will use the following steps:

1. compute average monthly excess returns
2. compute standard deviation of monthly excess returns
3. compute monthly Sharpe Ratios for the chosen stocks
4. annualise the computed monthly Sharpe Ratios

### Step 1: computing average excess returns

First we need to calculate the average of the monthly `excess_returns` computed in Step 8 over the analysed 5-year period. The calculated measure will tell us how much more or less the investment yields per month, on average, compared to the T-bill asset.

In [ ]:
# calculate the average of monthly excess_returns 
JPM_avg_exc = merged_data_tidy['JPM_exc'].mean()
MSFT_avg_exc = merged_data_tidy['MSFT_exc'].mean()

### Step 2: computing standard deviation of excess returns

Next, we calculate the standard deviation of the `excess_returns` computed in Spet 8. This measure captures the amount of total risk an investment in the stocks implies compared to an investment in the T-bill asset.

In [ ]:
# calculate the standard deviations
JPM_std_exc = merged_data_tidy['JPM_exc'].std()
MSFT_std_exc = merged_data_tidy['MSFT_exc'].std()

### Step 3: computing monthly Sharpe Ratios

Once the average monthly excess returns and the standard deviation of excess returns have been computed, we can calculate the Sharpe ratios for each stock as:

&nbsp;

$$Sharpe Ratio  = \frac{E|R_p - R_{rf}|}{\sqrt{var(R_p - R_{rf})}}$$

&nbsp;




In [ ]:
# calculate the Sharpe ratios
sharpe_month_JPM = JPM_avg_exc/JPM_std_exc
sharpe_month_MSFT = MSFT_avg_exc/MSFT_std_exc

### Step 4: annualising monthly Sharpe Ratios

The final step is to annualise the computed Sharpe ratios to put the numbers in a more useful one-year context, as follows:

&nbsp;

$$Sharpe Ratio_{AN} = Sharpe Ratio_M \times \sqrt{12}$$

&nbsp;

The higher the Sharpe Ratio, the better the asset’s historical risk-adjusted performance. The Sharpe Ratio is used widely to compare and rank portfolio managers with similar strategies on the basis of excess return each portfolio achieved per unit of total risk.


In [ ]:
# introduce an adjustment (annual) factor
annual_factor = np.sqrt(12)

# annualise the Sharpe ratios by applying the adjustment factor
sharpe_JPM = sharpe_month_JPM*annual_factor
sharpe_MSFT = sharpe_month_MSFT*annual_factor

# reporting the Sharpe ratios
print(sharpe_JPM, sharpe_MSFT)

## 10. Conclusion
Given the two Sharpe ratios, which investment should you go for? When choosing among investment alternatives that offer different levels of returns and risks, the Sharpe Ratio is used to adjust the investment returns for risk. 